In [16]:
import csv
from transformers import pipeline

def read_csv_data(filepath):
    with open(filepath, 'r') as file:
        # Use DictReader to read the CSV file, automatically using the first row for column names
        reader = csv.DictReader(file)
        data = [row for row in reader]

    #get rid of the entire empty column or row
    data = [d for d in data if any(d.values())]

    # Extract column names from the CSV
    column_names = reader.fieldnames

    # Create a dictionary to hold lists for each column
    columns = {name: [] for name in column_names}

    # Populate the lists with data from each column
    for row in data:
        for name in column_names:
            columns[name].append(row[name])

    # Convert the dictionary into separate lists
    lists = [columns[name] for name in column_names]
    return lists, column_names

# Use the function to read the CSV data
csv_path = ""

# Get the name between the last "/" and the second last "/" of csv_path
task_name = csv_path.split("/")[-2]
# task_name = "Political"

lists, column_names = read_csv_data(csv_path)

print(column_names)

models = {
    "Sentiment": {"model": "cardiffnlp/twitter-roberta-base-sentiment-latest", "label_len": 3},
    "Emotions": {"model": "SamLowe/roberta-base-go_emotions", "label_len": 5},
    "Gender": {"model": "padmajabfrl/Gender-Classification", "label_len": 2},
    "Irony": {"model": "cardiffnlp/twitter-roberta-base-irony", "label_len": 2},
    "Complexity": {"model": "krupper/text-complexity-classification", "label_len": 3},
    "Formality": {"model": "s-nlp/roberta-base-formality-ranker", "label_len": 2},
    "Language": {"model": "papluca/xlm-roberta-base-language-detection", "label_len": 3},
    "Politics": {"model": "m-newhauser/distilbert-political-tweets", "label_len": 2},
}

# Get the model and label_len based on the task name
model = models.get(task_name, {}).get("model")
label_len = models.get(task_name, {}).get("label_len")

# Check if the model exists for the given task
if model is None:
    print("No model found for the task:", task_name)
else:
    # Initialize the pipeline with the selected model
    pipeline = pipeline(model=model, top_k=None)
#iterate through the list of instructions and print the sentiment
for i in range(1,label_len+1):
    print()
    mistakes = []
    label=column_names[i]
    scores=[]
    num_labels=0
    for j in range(40):
        output= pipeline(lists[i][j])
        # print(lists[i][j])
        # print(output)
        highest_score = max(output[0], key=lambda x: x['score'])
        predicted_label = highest_score['label']
        # Iterate through the output list and find the score where the label equals to the given label
        for item in output[0]:
            if item['label'] == label:
                predicted_score = item['score']
                break        
        scores.append(predicted_score)
        if predicted_label==label:
            num_labels+=1
        else:
            # print()
            print("Question:", lists[0][j])
            # print("Actual label: ", label)
            # print("Predicted label: ", predicted_label)
            # print("Current Response: ", lists[i][j])
            mistakes.append(lists[i][j])
        # else:
        #     print("Predicted label: ", predicted_label)
        #     print("Actual label: ", label)
    print(label)
    # print("Average score: ", sum(scores)/len(scores))
    print("Percentage of correctly classified: ", num_labels/len(scores))
    if num_labels==0:
        # print(lists[i][j])
        print("No labels were correctly classified, condsider recheck the labels. The possible labels are: ", output)
        break

['\ufeffQuestion', 'Republican', 'Democrat']



Question: Are there any construction sites visible in the image?
Question: What type of buildings are seen in the background?
Question: Is the basketball court elevated or at ground level?
Question: What material is the railing made of?
Question: Is there greenery in the scene?
Question: What material does the railing appear to be made of?
Question: Are the buildings modern or historic in appearance?
Question: Are there visible yellow taxis in the background?
Republican
Percentage of correctly classified:  0.8

Question: Are the streets in the background crowded or relatively empty?
Question: Are there any advertisements or signs visible in the image?
Democrat
Percentage of correctly classified:  0.95
